In [8]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.special import softmax
import csv
import urllib.request

# Load CSV file and create DataFrame of Discord chat


In [75]:
data = pd.read_csv("DiscordData(aug).csv")
data.head()

,AuthorID,Author,Date,Content,Attachments,Reactions
0,370963041141129226,Daiigr#7964,02-Jul-20 08:53 PM,who wants to play something?,NaN,NaN
1,607796009707372554,Emo Knightmare#5231,02-Jul-20 10:17 PM,Sorry fam no gaming after 8 PM courtesy of my ...,NaN,NaN
2,370963041141129226,Daiigr#7964,03-Jul-20 12:26 AM,Oof,NaN,NaN
3,537937247891357697,The Dark Soul#7970,03-Jul-20 12:31 AM,big sad,NaN,NaN
4,503221991717863424,LET ME IN!!!#0749,03-Jul-20 01:14 AM,If anyone here plays DBD and wants to have som...,NaN,NaN


# Filtering Message for Use for Sentiment 
## removing bot messages

In [45]:
cleaned_data = data

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Rythm#3722'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Loli Police#5803'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Cool Frog Facts #frogs#0000'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Minecaft Bot#1886'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Dank Memer#5192' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Simple Poll#9879' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'UnoBot#6134' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'MEE6#4876' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Daiigr AI#4911' ].index)


print(cleaned_data['Author'].unique())

print(len(data))
print(len(cleaned_data))

['Daiigr#7964' 'Emo Knightmare#5231' 'The Dark Soul#7970'
 'LET ME IN!!!#0749' 'PETTAPLAY#7395' 'kima#1596' 'haani_05#5594'
 'Deleted User#0000' 'existential.duck#5117' 'Eeugh#0033'
 'Master chief#7460' 'KyroGaming#1987' 'soumar#3751' 'Iann#4893'
 'Haruka Sakura#2591' 'schrimpf#2940' 'Dwonks#8334' 'hagha#2602'
 'carlitos#1736' 'rY4n#8975' 'seafox2205#6878' 'Father#2152'
 'XxEXO_TROOPERxX#8603' "I'll do my best#9945" 'Peachymango11#3226'
 'lana#8237' 'sofia719462#2756' '(G)raveyard #dear-god#0000' 'Son#9990'
 'Lord Lazinga#7942' 'neurotell#9825' 'i have name#0161' 'Caitlin#6324']
187600
184938


# Preprocess text (username and link placeholders)


In [11]:
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
emoji, emotion, hate, irony, offensive, sentiment
stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary


In [12]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

# download label mapping

In [14]:
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [15]:
def listToString(instr):
        emptystr=""
        for ele in instr: 
            emptystr += ele + " "
        return emptystr

# Sampling Dataset and Averaging Sentiment

## Exploring our Data 

lets look how many messages each user has sent in this discord server. later on we will use this data to see if there is any corrolation between average sentiment and user messages

In [74]:
dugansToxicMessage = 'Children Death Slaves kill suicide'
text = dugansToxicMessage
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
score = softmax(scores)
print(score)

[0.96235037 0.03444532 0.00320427]


In [55]:
shittypeople = []
for Author in tqdm(data['Author'].unique()):
    totalCharactersSent = 0
    usermessage = data[data['Author'] == Author]
    usermessage = usermessage['Content'].copy()
    usermessage = usermessage.values
    for message in usermessage:
        totalCharactersSent += len(str(message))
    array = [Author, totalCharactersSent]
    shittypeople.append(array)

print(shittypeople)


100%|██████████| 42/42 [00:00<00:00, 137.60it/s]

[['Daiigr#7964', 324422], ['Emo Knightmare#5231', 216569], ['The Dark Soul#7970', 438301], ['LET ME IN!!!#0749', 176232], ['PETTAPLAY#7395', 1402278], ['kima#1596', 516], ['haani_05#5594', 11812], ['Deleted User#0000', 266713], ['existential.duck#5117', 225664], ['Eeugh#0033', 105], ['Master chief#7460', 7660], ['KyroGaming#1987', 41], ['soumar#3751', 269466], ['Iann#4893', 196510], ['Haruka Sakura#2591', 3493], ['schrimpf#2940', 106346], ['Dwonks#8334', 149145], ['Rythm#3722', 41893], ['hagha#2602', 73], ['carlitos#1736', 477115], ['rY4n#8975', 20], ['seafox2205#6878', 653330], ['MEE6#4876', 1289], ['Daiigr AI#4911', 4617], ['Father#2152', 702457], ['XxEXO_TROOPERxX#8603', 166], ["I'll do my best#9945", 53317], ['Simple Poll#9879', 113], ['Peachymango11#3226', 901], ['Minecaft Bot#1886', 8359], ['lana#8237', 140], ['sofia719462#2756', 739], ['UnoBot#6134', 18], ['(G)raveyard #dear-god#0000', 1391], ['Son#9990', 344], ['Loli Police#5803', 31055], ['Dank Memer#5192', 1435], ['Lord Lazin

In [56]:
import plotly.express as px

df = pd.DataFrame(shittypeople, columns = ['Author','Size'])
df.sort_values(by=['Size'], inplace=True)

fig = px.bar(df, x='Author', y='Size',
                template = "plotly_dark",
                color_discrete_sequence=px.colors.qualitative.Bold, title='Average Sentiment for Each User')
fig.show()

In [21]:
NumberOfMessages = []
avgUserScore = []
    
for Author in tqdm(cleaned_data['Author'].unique()):
    
    usermessage = cleaned_data[cleaned_data['Author'] == Author ]
    try:
     usermessage =  usermessage.sample(n=100)
    except:
        usermessage = usermessage
    #convert from dataframe to array of only the content of the message   
    UserContent = usermessage['Content'].copy()
    UserContent = UserContent.values
    #make sure all objects are strings and not any other type
    StrUserContent = []
    for line in UserContent:
        StrUserContent.append(str(line))

    Scores_arr = []
    for currentline in StrUserContent:
        try:
            text = currentline
            text = preprocess(text)
            encoded_input = tokenizer(text, return_tensors='pt')
            output = model(**encoded_input)
            scores = output[0][0].detach().numpy()
            Scores_arr.append(softmax(scores))
        except:
            pass

    
    totalScore = 0
    avgScore = 0
    #calcuate the average between the list of scores calulated for that user
    for i in Scores_arr:
        totalScore += i
    
    avgScore = totalScore/len(Scores_arr)
    avgUserScore.append([Author,avgScore[0],avgScore[1],avgScore[2],len(Scores_arr)])
#print array of avg user scores for all users
print(avgUserScore)

# show the Dataframe

100%|██████████| 33/33 [01:53<00:00,  3.44s/it]

[['Daiigr#7964', 0.26648533, 0.51279277, 0.22072193, 100], ['Emo Knightmare#5231', 0.31001696, 0.481238, 0.20874508, 100], ['The Dark Soul#7970', 0.27249047, 0.5343359, 0.19317345, 100], ['LET ME IN!!!#0749', 0.30158666, 0.48952883, 0.2088845, 100], ['PETTAPLAY#7395', 0.22808431, 0.5430755, 0.22884029, 100], ['kima#1596', 0.49690196, 0.3882993, 0.11479878, 15], ['haani_05#5594', 0.26872626, 0.53586394, 0.19540983, 100], ['Deleted User#0000', 0.34954157, 0.4863228, 0.16413563, 100], ['existential.duck#5117', 0.36971456, 0.45768523, 0.17260031, 99], ['Eeugh#0033', 0.22288015, 0.5777658, 0.199354, 9], ['Master chief#7460', 0.25843862, 0.5549174, 0.18664394, 100], ['KyroGaming#1987', 0.54242563, 0.41629434, 0.04127997, 2], ['soumar#3751', 0.30590144, 0.4841239, 0.20997438, 100], ['Iann#4893', 0.24527402, 0.60140115, 0.15332486, 100], ['Haruka Sakura#2591', 0.21577503, 0.5691975, 0.21502742, 100], ['schrimpf#2940', 0.27606472, 0.56762785, 0.15630749, 100], ['Dwonks#8334', 0.2708892, 0.51627

In [24]:
import plotly.express as px
df = pd.DataFrame(avgUserScore, columns = ['Author','Negative','Neutral','Possitive','SampleSize'])
df.sort_values(by=['Negative'], inplace=True)

fig = px.bar(df, x='Author', y=['Negative','Neutral','Possitive'],
                template = "plotly_dark",
                color_discrete_sequence=px.colors.qualitative.Bold, title='Average Sentiment for Each User')
fig.show()

## is there a trend between sample size and the extremity of the avg sentiment values?

In [23]:
# x and y given as DataFrame columns
import plotly.express as px
fig = px.scatter(df, x='SampleSize', y=['Negative','Neutral','Possitive'],trendline="lowess",trendline_options=dict(frac=0.4),template = "plotly_dark", 
title='sample size against average values of each sentiment')
fig.show()

/Users/daniel/miniforge3/envs/GPT/lib/python3.9/site-packages/statsmodels/nonparametric/smoothers_lowess.py:227: RuntimeWarning:

invalid value encountered in divide

/Users/daniel/miniforge3/envs/GPT/lib/python3.9/site-packages/statsmodels/nonparametric/smoothers_lowess.py:227: RuntimeWarning:

invalid value encountered in divide

/Users/daniel/miniforge3/envs/GPT/lib/python3.9/site-packages/statsmodels/nonparametric/smoothers_lowess.py:227: RuntimeWarning:

invalid value encountered in divide



In [20]:
import plotly.express as px
df.sort_values(by=['Neutral'], inplace=False)
fig = px.bar(df, x='Author', y=['Neutral'],template = "plotly_dark",color_discrete_sequence=px.colors.qualitative.Bold, title='nutrality')
fig.show()